In [6]:
pip install youtube-transcript-api

Note: you may need to restart the kernel to use updated packages.


In [74]:
from langchain_community.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import Ollama
from langchain.schema import Document
import json
from tqdm import tqdm
import pickle
from langchain.schema import Document
from collections import defaultdict
import time
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
#Done loading all pdf's, now time to get a link of all youtube videos that we want to scrape

In [ ]:
"""
NOTE - Load all youtube transcriptions first, then pdf files.
"""

In [ ]:
#Use google colab to load playlist urls, or the chrome extension mentioned in save_youtube_url notebook

In [ ]:
def load_youtube_video_transcripts(urls, sleeptime=5):
    youtube_video_transcripts = []
    success_urls = []
    failed_urls = []
    for url in tqdm(urls, desc="Loading YouTube Transcripts"):
        try:
            loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
            docs = loader.load()
            youtube_video_transcripts.extend(docs)
            success_urls.append(url)
        except Exception as e:
            print(f"\n❌ Failed to load {url}: {e}")
            failed_urls.append(url)
        time.sleep(sleeptime)  # Optional delay to avoid throttling
    
    # Summary
    print(f"\n✅ Loaded {len(success_urls)} transcripts successfully.")
    print(f"❌ Failed to load {len(failed_urls)} videos.")
        
    return youtube_video_transcripts, success_urls, failed_urls

In [19]:
# Load video URLs from file
with open("youtube_urls.txt", "r") as f:
    video_urls = [line.strip() for line in f if line.strip()]
    video_urls = list(set(video_urls))  #Getting rid of all the duplicates!

In [21]:
# Run the loader
youtube_video_transcripts, success_urls, failed_urls = load_youtube_video_transcripts(video_urls)

Loading YouTube Transcripts:   0%|          | 0/242 [00:00<?, ?it/s]


❌ Failed to load https://www.youtube.com/watch?v=m-7VjocC76M&list=PLjkaUn6QNTZSHfomCtXWV_hp0AZo_d_fa&index=15&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:   2%|▏         | 4/242 [00:24<24:42,  6.23s/it]


❌ Failed to load https://www.youtube.com/watch?v=MD5-HByRxoA&list=PLjkaUn6QNTZQtQnllF2ecgXpCPd33QfF2&index=6&t=1s&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:   5%|▍         | 11/242 [01:09<24:31,  6.37s/it]

...


❌ Failed to load https://www.youtube.com/watch?v=kW2vDMAmlPI&list=PLjkaUn6QNTZT4pNemi-7M5xFRfkuw3z-Z&index=59&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:  99%|█████████▉| 239/242 [26:12<00:20,  6.75s/it]


❌ Failed to load https://www.youtube.com/watch?v=MwhsM7qijy8&list=PLjkaUn6QNTZRvGqHusQcmB3tIOv1rD10_&index=48&pp=iAQB0gcJCY0JAYcqIYzv: no element found: line 1, column 0


Loading YouTube Transcripts: 100%|██████████| 242/242 [26:31<00:00,  6.58s/it]


✅ Loaded 187 transcripts successfully.
❌ Failed to load 55 videos.


In [ ]:
#Yep, there is a rate limiting issue, add 5 seconds or even more maybe 10 secodns!, or maybe big videos fail and small ones dont!

In [22]:
success_urls[0:5]

['https://www.youtube.com/watch?v=bgBIO6nZawg&list=PLjkaUn6QNTZSHfomCtXWV_hp0AZo_d_fa&index=3&pp=iAQB',
 'https://www.youtube.com/watch?v=sSEm3qJUh9s&list=PLjkaUn6QNTZRvGqHusQcmB3tIOv1rD10_&index=6&pp=iAQB',
 'https://www.youtube.com/watch?v=2tHlHWgDRdQ&list=PLjkaUn6QNTZT4pNemi-7M5xFRfkuw3z-Z&index=56&pp=iAQB',
 'https://www.youtube.com/watch?v=Jm91dWD7LHg&list=PLRXjv_N1fWrP4QmCkYtXLDqiNTc7UsOr_&index=3&pp=iAQB',
 'https://www.youtube.com/watch?v=INm4U2S7Vu8&list=PLjkaUn6QNTZQFCBYrF01ZXjWgBFhTgalQ&index=6&pp=iAQB']

In [23]:
#Lets check if theres a difference between URL;s of successful and failed loads
failed_urls[0:5]

['https://www.youtube.com/watch?v=m-7VjocC76M&list=PLjkaUn6QNTZSHfomCtXWV_hp0AZo_d_fa&index=15&pp=iAQB',
 'https://www.youtube.com/watch?v=MD5-HByRxoA&list=PLjkaUn6QNTZQtQnllF2ecgXpCPd33QfF2&index=6&t=1s&pp=iAQB',
 'https://www.youtube.com/watch?v=LVM89ik-7Kw&list=PLjkaUn6QNTZQmeMz0CnyUaeO_P4v4bq1W&index=10&pp=iAQB',
 'https://www.youtube.com/watch?v=Day0yToqeco&list=PLjkaUn6QNTZQFCBYrF01ZXjWgBFhTgalQ&index=2&pp=iAQB',
 'https://www.youtube.com/watch?v=_gcqwupsza8&list=PLjkaUn6QNTZT4pNemi-7M5xFRfkuw3z-Z&index=58&pp=iAQB']

In [27]:
# Run the loader
youtube_video_transcripts_stage2, success_urls_stage2 , failed_urls_stage2 = load_youtube_video_transcripts(failed_urls, sleeptime=10)

Loading YouTube Transcripts:  11%|█         | 6/55 [01:07<09:11, 11.25s/it]


❌ Failed to load https://www.youtube.com/watch?v=RVbvhPGFi6E&list=PLjkaUn6QNTZTd2rKxXdy-_I0JDHlwdy6j&index=46&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:  15%|█▍        | 8/55 [01:30<08:49, 11.26s/it]


❌ Failed to load https://www.youtube.com/watch?v=GIRkQQHzsxI&list=PLjkaUn6QNTZTd2rKxXdy-_I0JDHlwdy6j&index=71&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:  35%|███▍      | 19/55 [03:33<06:45, 11.26s/it]

...


❌ Failed to load https://www.youtube.com/watch?v=sBJppqCeFGI&list=PLjkaUn6QNTZTrJTfCYC-BfeVJVL9_nz6Z&index=5&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:  96%|█████████▋| 53/55 [10:10<00:23, 11.57s/it]


❌ Failed to load https://www.youtube.com/watch?v=kW2vDMAmlPI&list=PLjkaUn6QNTZT4pNemi-7M5xFRfkuw3z-Z&index=59&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts: 100%|██████████| 55/55 [10:33<00:00, 11.52s/it]


✅ Loaded 47 transcripts successfully.
❌ Failed to load 8 videos.


In [ ]:
# Run the loader, Final time, no more runs. whatever fails now fails.
youtube_video_transcripts_stage3, success_urls_stage3 , failed_urls_stage3 = load_youtube_video_transcripts(failed_urls_stage2, sleeptime=25)

Loading YouTube Transcripts:   0%|          | 0/8 [00:00<?, ?it/s]


❌ Failed to load https://www.youtube.com/watch?v=RVbvhPGFi6E&list=PLjkaUn6QNTZTd2rKxXdy-_I0JDHlwdy6j&index=46&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts:  62%|██████▎   | 5/8 [02:12<01:19, 26.59s/it]


❌ Failed to load https://www.youtube.com/watch?v=rp1PzCxj3eU&list=PLjkaUn6QNTZQFCBYrF01ZXjWgBFhTgalQ&index=5&pp=iAQB: no element found: line 1, column 0


Loading YouTube Transcripts: 100%|██████████| 8/8 [03:32<00:00, 26.53s/it]


✅ Loaded 6 transcripts successfully.
❌ Failed to load 2 videos.


In [30]:
failed_urls_stage2

['https://www.youtube.com/watch?v=RVbvhPGFi6E&list=PLjkaUn6QNTZTd2rKxXdy-_I0JDHlwdy6j&index=46&pp=iAQB',
 'https://www.youtube.com/watch?v=GIRkQQHzsxI&list=PLjkaUn6QNTZTd2rKxXdy-_I0JDHlwdy6j&index=71&pp=iAQB',
 'https://www.youtube.com/watch?v=y3T4GqoH0Us&list=PLjkaUn6QNTZRvGqHusQcmB3tIOv1rD10_&index=32&pp=iAQB0gcJCY0JAYcqIYzv',
 'https://www.youtube.com/watch?v=k-3PoOT4vOM&list=PLjkaUn6QNTZTd2rKxXdy-_I0JDHlwdy6j&index=37&pp=iAQB0gcJCY0JAYcqIYzv',
 'https://www.youtube.com/watch?v=PArqypiHcyQ&list=PLjkaUn6QNTZT4pNemi-7M5xFRfkuw3z-Z&index=49&pp=iAQB',
 'https://www.youtube.com/watch?v=rp1PzCxj3eU&list=PLjkaUn6QNTZQFCBYrF01ZXjWgBFhTgalQ&index=5&pp=iAQB',
 'https://www.youtube.com/watch?v=sBJppqCeFGI&list=PLjkaUn6QNTZTrJTfCYC-BfeVJVL9_nz6Z&index=5&pp=iAQB',
 'https://www.youtube.com/watch?v=kW2vDMAmlPI&list=PLjkaUn6QNTZT4pNemi-7M5xFRfkuw3z-Z&index=59&pp=iAQB']

In [29]:
youtube_video_transcripts_stage3

[Document(page_content="productivity is how much money you get out for the time you put in I know this because I went from $0 at age 23 to $100 million plus net worth at age 31 simply by better investing my time and I'm going to teach you how I do it so that hopefully you can do it and I'm even going to show you my calendar at the end of the video and walk you through step by step how I get good Returns on my time have you ever had someone reach out to you and say hey God minute hey got 10 hey let me introduce you to so and so they're going to be in town how does 10 a.m. work a normal exchange like that you might say yeah I've Got 5 minutes or sure I can meet so and so for coffee the problem is that for one specific type of worker that five-minute meeting blows their entire morning because they can't do the development work they need they can't edit the video they can't write the book chapter because it disrupts their workflow on the other hand there's another type of worker where for 

In [34]:
filepaths = ["/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/Alex Hormozi 100 million leads.pdf",
             "/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/Alex Hormozi 100m Offers.pdf",
             "/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/DotCom Secrets Russel Brunson.pdf",
             "/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/Expert-Secrets-Russel Brunson.pdf",
             "/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/Russel Brunson Lead Funnels.pdf",
             "/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/Sabri Suby Sell like crazy.pdf",
             "/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/pdf_files/Traffic Secrets Russell Brunson.pdf"]

In [35]:
async def read_pdfs_into_pages(filepaths):
    pages = []
    for filepath in filepaths:
        loader = PyPDFLoader(filepath)
        async for page in loader.alazy_load():
            pages.append(page)
    return pages

In [36]:
all_marketing_pdf_pages = await read_pdfs_into_pages(filepaths)

In [ ]:
#mixing all docs and saving locally

In [39]:
all_youtube_transcriptions = youtube_video_transcripts + youtube_video_transcripts_stage2 + youtube_video_transcripts_stage3

In [42]:
# Save each list of Documents separately
with open("youtube_transcriptions.pkl", "wb") as f:
    pickle.dump(all_youtube_transcriptions, f)

with open("marketing_pdfs.pkl", "wb") as f:
    pickle.dump(all_marketing_pdf_pages, f)

In [44]:
with open("/Users/hitesh.modi/Desktop/Kinda Personal/LLM Finetuning Expert/youtube_channel_mappings.json", "r", encoding="utf-8") as f:
    youtube_channel_mappings = json.load(f)

In [ ]:
"""
WE ARE HERE!
"""

In [ ]:
all_youtube_transcriptions_test2 = all_youtube_transcriptions.copy()

In [56]:
for doc in all_youtube_transcriptions:
    video_id = doc.metadata.get("source")
    if video_id in youtube_channel_mappings:
        doc.metadata["channel"] = f"{youtube_channel_mappings[video_id]} Youtube Channel"  # Add a new key for clarity

In [ ]:
#Lets scrape andrew tate stuff as well and add that as well to the Doc

In [ ]:
#We need to add name of books to books metadata and name of youtube channels to youtube metadata. Then add the metadata and also change the prompt to keep sources / metadata.

In [59]:
# Run the loader, Final time, no more runs. whatever fails now fails.
Andrew_tate_hustleruniversity_transcription, Andrew_tate_hustleruniversity_success , Andrew_tate_hustleruniversity_failed = load_youtube_video_transcripts(["https://www.youtube.com/watch?v=owoI9qhOzyM"], sleeptime=5)

Loading YouTube Transcripts: 100%|██████████| 1/1 [00:06<00:00,  6.77s/it]


✅ Loaded 1 transcripts successfully.
❌ Failed to load 0 videos.


In [61]:
for doc in Andrew_tate_hustleruniversity_transcription:
    doc.metadata["channel"] = f"Andrew Tate Hustler University Old"


In [62]:
Andrew_tate_hustleruniversity_transcription

[Document(page_content="welcome to the smartest decision you've ever made husters University this is the smartest thing you've ever done if you're sitting there wondering you should already know but if you're sitting there wondering is this going to be worth it or not Buckle the [\xa0__\xa0] up this is the smartest thing you've ever done because this is the first time besides the PHD course where I'm going to lay down all of my knowledge in the world about a particular subject PhD course was girls this is going to be about business everything I know about making money so the way this came around was I was convincing yet another person to not University I have convinced well I don't know if I've actually convinced them but I often say the biggest mistake people make in their lives is traditional education it's a waste of time it's a scam and I always tell people why it's a waste of time why it's a scam like there's the double inflation principle that I coined myself the fact that more d

In [63]:
all_youtube_transcriptions = all_youtube_transcriptions + Andrew_tate_hustleruniversity_transcription

In [65]:
len(youtube_video_transcripts) + len(youtube_video_transcripts_stage2) + len(youtube_video_transcripts_stage3)

239

In [64]:
len(all_youtube_transcriptions)

240

In [70]:
import os

for doc in all_marketing_pdf_pages:
    full_path = doc.metadata.get("source", "")
    # Extract just the file name, remove the extension
    base_name = os.path.basename(full_path).replace(".pdf", "")
    # Update the metadata
    doc.metadata["source"] = base_name

    # Optional: remove page number if not needed
    if "page" in doc.metadata:
        del doc.metadata["page"]


In [ ]:
"""
Now, each Document (like one page of a PDF) is treated as its own input by default.

So:

If your PDF pages are stored as individual LangChain Document objects, and you run the splitter on the full list, the splitter resets for each page.

So, we have to combine each document into 1 doc first
"""

In [76]:
async def read_and_merge_pdfs(filepaths):
    merged_documents = []

    for filepath in filepaths:
        loader = PyPDFLoader(filepath)
        pages = []
        async for page in loader.alazy_load():
            pages.append(page.page_content)
        
        combined_text = "\n".join(pages)
        pdf_title = filepath.split("/")[-1].replace(".pdf", "")  # Clean file name
        
        merged_documents.append(Document(
            page_content=combined_text,
            metadata={"source": pdf_title}
        ))

    return merged_documents

In [77]:
updated_all_marketing_pdf_pages = await read_and_merge_pdfs(filepaths)

In [88]:
updated_all_marketing_pdf_pages[0].page_content

'\n \nA c q u i s i t i o n . c o m  V o l u m e  I I\n \n$100M Leads\n \nH o w  t o  G e t  S t r a n g e r s  T o\nW a n t  T o  B u y  Y o u r  S t u f f\n \n \nA l e x  H o r m o z i\nCopyright © 2023 by Alex Hormozi\n \nAll rights reserved. No part of this publication may be reproduced,\ndistributed, or transmitted in any form or by any means, including\nphotocopying, recording, or other electronic or mechanical\nmethods, without the prior written permission of the publisher ,\nexcept in the case of brief questions embodied in critical reviews\nand certain other noncommercial uses permitted by copyright law .\nFor permission requests, write to the publisher at the address\nbelow .\nAcquisition.com\n7710 N FM 620, Building 13C, Suite 100,\nAustin, T exas 78726\nGuiding Principles\n \n \n \nD o  m o r e .\n \n \nT h a n k  Y o u s\n \n \nT o T r evor:\nThank you for your true friendship. Thank you for your tireless\nef fort to extract the ideas out of my head. And, for your continue

In [89]:
# Save each list of Documents separately
with open("youtube_transcriptions.pkl", "wb") as f:
    pickle.dump(all_youtube_transcriptions, f)

with open("marketing_pdfs.pkl", "wb") as f:
    pickle.dump(updated_all_marketing_pdf_pages, f)

In [90]:
def split_pages(pages):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=300,
    separators=["\n\n", "\n", " ", ""]
    )

    docs = text_splitter.split_documents(pages)
    return docs

In [91]:
Full_marketing_specialist_material = updated_all_marketing_pdf_pages + all_youtube_transcriptions
pages = split_pages(Full_marketing_specialist_material)

In [97]:
with open("all_marketing_material.pkl", "wb") as f:
    pickle.dump(pages, f)

In [92]:
pages[0:5]

[Document(page_content='A c q u i s i t i o n . c o m  V o l u m e  I I\n \n$100M Leads\n \nH o w  t o  G e t  S t r a n g e r s  T o\nW a n t  T o  B u y  Y o u r  S t u f f\n \n \nA l e x  H o r m o z i\nCopyright © 2023 by Alex Hormozi\n \nAll rights reserved. No part of this publication may be reproduced,\ndistributed, or transmitted in any form or by any means, including\nphotocopying, recording, or other electronic or mechanical\nmethods, without the prior written permission of the publisher ,\nexcept in the case of brief questions embodied in critical reviews\nand certain other noncommercial uses permitted by copyright law .\nFor permission requests, write to the publisher at the address\nbelow .\nAcquisition.com\n7710 N FM 620, Building 13C, Suite 100,\nAustin, T exas 78726\nGuiding Principles\n \n \n \nD o  m o r e .\n \n \nT h a n k  Y o u s\n \n \nT o T r evor:\nThank you for your true friendship. Thank you for your tireless\nef fort to extract the ideas out of my head. And,

In [98]:
llm = Ollama(model="gemma3:4b")

In [99]:
def local_llm(prompt: str, llm) -> str:
    """
    Sends a prompt to the LangChain Ollama LLM instance and returns the response.
    """
    try:
        return llm.invoke(prompt).strip()
    except Exception as e:
        print("Error invoking model:", e)
        return ""


In [101]:
def process_chunk_to_alpaca(doc: Document, llm) -> dict:
    # Extract metadata from the LangChain Document
    source_name = doc.metadata.get("source", "Unknown Name")

    # Inject metadata into prompt
    instruction_with_metadata = f"""
You are a business assistant analyzing raw business content from the following source:
SOURCE NAME: {source_name}

Your task is to extract the following from the provided transcript:
1. Frameworks (e.g., naming, advertising, validation models).
2. Bullet points for key ideas or steps.
3. Q&A (any implied or stated questions with answers).
4. Case Examples or stories.
5. Copywriting formulas (AIDA, PAS, etc.)
6. Classify this content into high-level topics: e.g., Naming, Ads, Psychology, Copywriting.
7. Convert suitable content into a step-by-step guide.

Return your output in clearly labeled sections, and only include sections with relevant content. Do not include a preamble.
""".strip()

    prompt = f"{instruction_with_metadata}\n\n{doc.page_content.strip()}"
    response = local_llm(prompt, llm)

    return {
        "instruction": instruction_with_metadata,
        "input": doc.page_content.strip(),
        "output": response,
        "metadata": doc.metadata
    }


In [106]:
len(pages)

6608

In [103]:
alpaca_data = []

for doc in pages[0:5]:  # each doc is a langchain Document
    alpaca_entry = process_chunk_to_alpaca(doc, llm)
    alpaca_data.append(alpaca_entry)

In [104]:
alpaca_data

[{'instruction': 'You are a business assistant analyzing raw business content from the following source:\nSOURCE NAME: Alex Hormozi 100 million leads\n\nYour task is to extract the following from the provided transcript:\n1. Frameworks (e.g., naming, advertising, validation models).\n2. Bullet points for key ideas or steps.\n3. Q&A (any implied or stated questions with answers).\n4. Case Examples or stories.\n5. Copywriting formulas (AIDA, PAS, etc.)\n6. Classify this content into high-level topics: e.g., Naming, Ads, Psychology, Copywriting.\n7. Convert suitable content into a step-by-step guide.\n\nReturn your output in clearly labeled sections, and only include sections with relevant content. Do not include a preamble.',
  'input': 'A c q u i s i t i o n . c o m  V o l u m e  I I\n \n$100M Leads\n \nH o w  t o  G e t  S t r a n g e r s  T o\nW a n t  T o  B u y  Y o u r  S t u f f\n \n \nA l e x  H o r m o z i\nCopyright © 2023 by Alex Hormozi\n \nAll rights reserved. No part of thi

In [ ]:
#This is where we will be finally processing everything through a LLM

In [ ]:
PROCESSED_FILE = "alpaca_processed.jsonl"
FAILED_FILE = "alpaca_failed.jsonl"
MAX_RETRIES = 3
RETRY_DELAY = 2  # seconds between retries

def load_jsonl_ids(filename):
    if not os.path.exists(filename):
        return set()
    with open(filename, "r") as f:
        return {json.loads(line).get("metadata", {}).get("source", "") + str(json.loads(line).get("metadata", {}).get("page", "")) for line in f}

def save_jsonl(filename, data):
    with open(filename, "a") as f:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def get_doc_id(doc: Document):
    source = doc.metadata.get("source", "")
    page = doc.metadata.get("page")

    if page is None:
        # Fallback to hashing part of the content if page is missing
        content_hash = str(abs(hash(doc.page_content[:50])))
        return f"{source}_hash_{content_hash}"

    return f"{source}_page_{page}"

def process_documents_with_retries(pages, llm):
    processed_ids = load_jsonl_ids(PROCESSED_FILE)
    failed_ids = load_jsonl_ids(FAILED_FILE)

    for doc in tqdm(pages, desc="Processing documents"):
        doc_id = get_doc_id(doc)

        if doc_id in processed_ids:
            continue

        retries = 0
        success = False

        while retries < MAX_RETRIES and not success:
            try:
                alpaca_entry = process_chunk_to_alpaca(doc, llm)
                save_jsonl(PROCESSED_FILE, alpaca_entry)
                success = True
            except Exception as e:
                retries += 1
                if retries < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    error_entry = {
                        "error": str(e),
                        "metadata": doc.metadata,
                        "input": doc.page_content[:500]  # preview of failed input
                    }
                    save_jsonl(FAILED_FILE, error_entry)
        time.sleep(25)

In [117]:
process_documents_with_retries(pages, llm)

Processing documents:   0%|          | 0/6608 [00:00<?, ?it/s]

Processing documents:  16%|█▋        | 1082/6608 [1:45:31<8:58:57,  5.85s/it] 


KeyboardInterrupt: 